In [1]:
import time
import getpass
import os
import sys
from tqdm.notebook import tqdm as log_progress

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

In [2]:
#define constants
USERNAME_ENV_VARIABLE = 'PIAZZA_UN'
PASSWORD_ENV_VARIABLE = 'PIAZZA_PSWD'

CHROMEDRIVER_PATH = 'C:/tools/chrome-driver/chromedriver.exe'

In [11]:
#reset environment variables (if needed)
os.environ[USERNAME_ENV_VARIABLE] = ''
os.environ[PASSWORD_ENV_VARIABLE] = ''

In [3]:
#get credentials
if not os.getenv(USERNAME_ENV_VARIABLE) or os.getenv(USERNAME_ENV_VARIABLE) == '':
    os.environ[USERNAME_ENV_VARIABLE] = input('Enter email for login: ')
if not os.getenv(PASSWORD_ENV_VARIABLE) or os.getenv(PASSWORD_ENV_VARIABLE) == '':
    os.environ[PASSWORD_ENV_VARIABLE] = getpass.getpass('Enter password for login: ')

Enter email for login:  luc528@lehigh.edu
Enter password for login:  ········


In [4]:
class_url = "https://piazza.com/class/m470f2rs65zff"

options = Options()
#options.add_argument('--headless')
options.add_argument("--incognito")
options.add_argument("--nogpu")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1280,1280")
options.add_argument("--no-sandbox")
options.add_argument("--enable-javascript")

driver = webdriver.Chrome(CHROMEDRIVER_PATH, options=options)

driver.get(class_url)

#login
print("Logging in...")
try:
    driver.find_element_by_name("email").send_keys(os.getenv(USERNAME_ENV_VARIABLE))
    driver.find_element_by_name("password").send_keys(os.getenv(PASSWORD_ENV_VARIABLE))
    driver.find_element(By.XPATH, '//button[text()="Log in"]').click()
except Exception as e:
    print("Error logging in!")
    print(e)
    sys.exit()

print("Logged in, beginning scrape...")
time.sleep(1) #wait for page to update

#fetch total number of posts
num_posts = int(driver.find_element(By.XPATH, '//*[@id="qanda-content"]/article[2]/div/div[1]/div[2]/div[2]/div[1]').text)

posts = []
'''
    posts [
        {
            "post": <post_content>,
            "replies": {
                'student': <text>,
                'instructor': <text>,
                'follow-up': <text>
            }
        },
    ]
'''
#TODO: organize by category (different json for each homework assignment)

#scrape all messages and responses
for post_num in log_progress(range(1, num_posts+1)):
    url = f'{class_url}/post/{post_num}'
    
    try:
        driver.get(url)

        post = driver.find_element(By.XPATH, '//*[@id="qaContentViewId"]/div/div/div/div[2]').text
        
        student_answer = driver.find_element(By.XPATH, '//*[@id="qanda-content"]/article[2]/div').text
        instructor_answer = driver.find_element(By.XPATH, '//*[@id="qanda-content"]/article[3]/div').text
        follow_up = driver.find_element(By.XPATH, '//*[@id="qanda-content"]/article[4]/div').text
        
        posts.append({})
        posts[-1]['post'] = post
        
        posts[-1]['replies'] = {}
        posts[-1]['replies']['student'] = student_answer
        posts[-1]['replies']['instructor'] = instructor_answer
        posts[-1]['replies']['follow-up'] = follow_up
    except:
        continue #just keep on goin
    
print(posts)
driver.close()

Logging in...
Logged in, beginning scrape...


  0%|          | 0/340 [00:00<?, ?it/s]

[{'post': 'Grade Bump for Sections\nAre there going to be grade bumps for groups 2 & 3?', 'replies': {'student': '', 'instructor': 'There will not be grade bumps for groups 2 & 3', 'follow-up': 'Start a new followup discussion'}}]
